In [1]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [3]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [4]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [5]:
#Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# 1- Service Quality Score
train['service_score'] = (
    train['Inflight entertainment'] +
    train['Seat comfort'] +
    train['On-board service'] +
    train['Leg room service'] +
    train['Cleanliness'] +
    train['Inflight wifi service'] +
    train['Inflight service'] +
    train['Checkin service'] +
    train['Food and drink']
)
test['service_score'] = (
    test['Inflight entertainment'] +
    test['Seat comfort'] +
    test['On-board service'] +
    test['Leg room service'] +
    test['Cleanliness'] +
    test['Inflight wifi service'] +
    test['Inflight service'] +
    test['Checkin service'] +
    test['Food and drink']
)

In [6]:
# 3- Travel Type
train['Travel_Class'] = train['Type of Travel']  + train['Class']
test['Travel_Class'] = test['Type of Travel']  + test['Class']

In [7]:
# 4- Loyalty or New Customer
train['Is_New_Customer'] = (train['Customer Type'] == 'disloyal Customer').astype(int)
test['Is_New_Customer'] = (test['Customer Type'] == 'disloyal Customer').astype(int)

In [8]:
# 5- Flight Duration
train['Flight Category'] = pd.cut(train['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])
test['Flight Category'] = pd.cut(test['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# فقط ستون‌های مؤثر رو نگه می‌داریم:
useful_features = [
    'Online boarding',
    'Inflight entertainment',
    'Seat comfort',
    'On-board service',
    'Leg room service',
    'Cleanliness',
    'Flight Distance',
    'Inflight wifi service',
    'Baggage handling',
    'Inflight service',
    'Checkin service',
    'Food and drink',
    'Ease of Online booking',
    'Age',
    'service_score',
    'Travel_Class',
    'Is_New_Customer',
    'Flight Category'
]

# به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# فقط ستون‌های موردنیاز رو برمی‌داریم
train = train[useful_features + categorical_cols + ['satisfaction']]
test = test[useful_features + categorical_cols]

In [10]:
train['satisfaction'].head()

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [12]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [13]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [14]:
train.isna().sum()

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers

# --- Hyperparameters ---
LEARNING_RATE = 0.0003
L2_REG = 1e-5
# Reduced dropout rates (tuned based on experience)
DROPOUT_RATES = [0.3, 0.2, 0.1, 0.05]
BATCH_SIZE = 64
EPOCHS = 100

# --- Assumption: X_train, y_train, X_valid, y_valid are already defined ---
# For example, these could come from your preprocessed and split data:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- Model Definition ---
inputs = Input(shape=(X_train.shape[1],))
x = Dense(384, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[0])(x)

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[1])(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[2])(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[3])(x)

outputs = Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# --- Compile Model ---
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# --- Callbacks ---
checkpoint_cb = ModelCheckpoint("model_962.keras",
                                  save_best_only=True,
                                  monitor='val_accuracy',  # Monitor accuracy improvements
                                  mode='max',
                                  verbose=1)
earlystop_cb = EarlyStopping(patience=10,
                             restore_best_weights=True,
                             monitor='val_accuracy',
                             mode='max',
                             verbose=1)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss',
                                 patience=3,
                                 factor=0.5,
                                 min_lr=1e-6,
                                 verbose=1)

# --- Train the Model ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers

# --- Example: Load your data and perform train/test split ---
# Assumed that you have a dataset and you split it into features (X) and target (y).
# Replace the following placeholders with your actual data loading/preprocessing.
# For example:
# data = pd.read_csv("your_data.csv")
# X = data.drop("target", axis=1).values
# y = data["target"].values
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Example placeholder arrays (replace these with your actual preprocessed arrays):
X_train = np.random.rand(800, 20)  # placeholder: 800 samples, 20 features
X_valid = np.random.rand(200, 20)
y_train = np.random.randint(0, 2, 800)  # binary target
y_valid = np.random.randint(0, 2, 200)

# --- Ensure data is numeric (convert to float32) ---
X_train = np.asarray(X_train).astype('float32')
X_valid = np.asarray(X_valid).astype('float32')
# For binary crossentropy, it's a good idea to work in float32 even for targets
y_train = np.asarray(y_train).astype('float32')
y_valid = np.asarray(y_valid).astype('float32')

# --- Revised Model Definition with L2 Regularization and Adjusted Dropout ---
LEARNING_RATE = 0.0003
L2_REG = 1e-5
# Adjusted dropout rates for improved performance
DROPOUT_RATES = [0.3, 0.2, 0.1, 0.05]

inputs = Input(shape=(X_train.shape[1],))
x = Dense(384, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[0])(x)

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[1])(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[2])(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(DROPOUT_RATES[3])(x)

outputs = Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# --- Compile Model ---
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# --- Define Callbacks ---
checkpoint_cb = ModelCheckpoint("model_962.keras", save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
earlystop_cb = EarlyStopping(patience=7, restore_best_weights=True, monitor='val_accuracy', mode='max')
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# --- Train ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [17]:
from sklearn.metrics import accuracy_score
# Evaluate
y_pred_probs = model.predict(X_valid)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Get model predictions (as probabilities)
y_val_pred_probs = model.predict(X_valid)

# Convert probabilities to class labels (0 or 1)
y_val_pred = (y_val_pred_probs > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_val_pred)

print(f"Validation Accuracy: {accuracy:.4f}")

# Step 1: Predict (assumes X_test is preprocessed)
y_test_pred_probs = model.predict(test)
y_test_pred = (y_test_pred_probs > 0.5).astype(int).flatten()

# Step 2: Map 0/1 to correct labels
label_map = {0: 'neutral or dissatisfied', 1: 'satisfied'}
y_test_labels = [label_map[val] for val in y_test_pred]
print(y_test_labels)
#

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Model Definition
inputs = Input(shape=(X_train.shape[1],))
x = Dense(384, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(64, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

outputs = Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint_cb = ModelCheckpoint("model_962.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=7, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [19]:
from sklearn.metrics import accuracy_score
# Evaluate
y_pred_probs = model.predict(X_valid)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Get model predictions (as probabilities)
y_val_pred_probs = model.predict(X_valid)

# Convert probabilities to class labels (0 or 1)
y_val_pred = (y_val_pred_probs > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_val_pred)

print(f"Validation Accuracy: {accuracy:.4f}")

# Step 1: Predict (assumes X_test is preprocessed)
y_test_pred_probs = model.predict(test)
y_test_pred = (y_test_pred_probs > 0.5).astype(int).flatten()

# Step 2: Map 0/1 to correct labels
label_map = {0: 'neutral or dissatisfied', 1: 'satisfied'}
y_test_labels = [label_map[val] for val in y_test_pred]
print(y_test_labels)
#

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Model Definition (2 layers)
inputs = Input(shape=(X_train.shape[1],))

# First hidden layer
x = Dense(128, kernel_initializer='he_normal')(inputs)  # Adjust number of neurons as needed
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)  # Adjust dropout rate as needed

# Output layer
outputs = Dense(1, activation='sigmoid')(x)  

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks (remain the same)
checkpoint_cb = ModelCheckpoint("model_962.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=7, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Train (remain the same)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [21]:
from sklearn.metrics import accuracy_score
# Evaluate
y_pred_probs = model.predict(X_valid)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Get model predictions (as probabilities)
y_val_pred_probs = model.predict(X_valid)

# Convert probabilities to class labels (0 or 1)
y_val_pred = (y_val_pred_probs > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_val_pred)

print(f"Validation Accuracy: {accuracy:.4f}")

# Step 1: Predict (assumes X_test is preprocessed)
y_test_pred_probs = model.predict(test)
y_test_pred = (y_test_pred_probs > 0.5).astype(int).flatten()

# Step 2: Map 0/1 to correct labels
label_map = {0: 'neutral or dissatisfied', 1: 'satisfied'}
y_test_labels = [label_map[val] for val in y_test_pred]
print(y_test_labels)
#

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Model Definition (2 layers with potential improvements)
inputs = Input(shape=(X_train.shape[1],))

# First hidden layer (with L2 regularization)
x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(inputs)  
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x) 

# Output layer
outputs = Dense(1, activation='sigmoid')(x)  

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile (with a lower learning rate)
optimizer = keras.optimizers.Adam(learning_rate=0.0001)  
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks (with increased patience)
checkpoint_cb = ModelCheckpoint("model_962.keras", save_best_only=True, monitor='val_accuracy') 
earlystop_cb = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy')  
reduce_lr_cb = ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.5, min_lr=1e-6, verbose=1) 

# Train (with more epochs if needed)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=200,  
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(256, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(256, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [25]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [26]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [27]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [28]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [29]:
#Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# 1- Service Quality Score
train['service_score'] = (
    train['Inflight entertainment'] +
    train['Seat comfort'] +
    train['On-board service'] +
    train['Leg room service'] +
    train['Cleanliness'] +
    train['Inflight wifi service'] +
    train['Inflight service'] +
    train['Checkin service'] +
    train['Food and drink']
)
test['service_score'] = (
    test['Inflight entertainment'] +
    test['Seat comfort'] +
    test['On-board service'] +
    test['Leg room service'] +
    test['Cleanliness'] +
    test['Inflight wifi service'] +
    test['Inflight service'] +
    test['Checkin service'] +
    test['Food and drink']
)

In [30]:
#2-Total Delay
train['Total Delay'] = train['Departure Delay in Minutes'] + train['Arrival Delay in Minutes']
train['Delay Category'] = pd.cut(train['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

#2-Total Delay
test['Total Delay'] = test['Departure Delay in Minutes'] + test['Arrival Delay in Minutes']
test['Delay Category'] = pd.cut(test['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

In [31]:
# 3- Travel Type
train['Travel_Class'] = train['Type of Travel']  + train['Class']
test['Travel_Class'] = test['Type of Travel']  + test['Class']

In [32]:
# 4- Loyalty or New Customer
train['Is_New_Customer'] = (train['Customer Type'] == 'disloyal Customer').astype(int)
test['Is_New_Customer'] = (test['Customer Type'] == 'disloyal Customer').astype(int)

In [33]:
# 5- Flight Duration
train['Flight Category'] = pd.cut(train['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])
test['Flight Category'] = pd.cut(test['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# فقط ستون‌های مؤثر رو نگه می‌داریم:
useful_features = [
    'Online boarding',
    'Inflight entertainment',
    'Seat comfort',
    'On-board service',
    'Leg room service',
    'Cleanliness',
    'Flight Distance',
    'Inflight wifi service',
    'Baggage handling',
    'Inflight service',
    'Checkin service',
    'Food and drink',
    'Ease of Online booking',
    'Age',
    'service_score',
    'Travel_Class',
    'Is_New_Customer',
    'Flight Category'
]

# به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# فقط ستون‌های موردنیاز رو برمی‌داریم
train = train[useful_features + categorical_cols + ['satisfaction']]
test = test[useful_features + categorical_cols]

In [35]:
train['satisfaction'].head()

In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [37]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [38]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [39]:
test.head()

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(256, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [44]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [45]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [46]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [47]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# فقط ستون‌های مؤثر رو نگه می‌داریم:
useful_features = [
    'Online boarding',
    'Inflight entertainment',
    'Seat comfort',
    'On-board service',
    'Leg room service',
    'Cleanliness',
    'Flight Distance',
    'Inflight wifi service',
    'Baggage handling',
    'Inflight service',
    'Checkin service',
    'Food and drink',
    'Ease of Online booking',
    'Age',
    'service_score',
    'Travel_Class',
    'Is_New_Customer',
    'Flight Category'
]

# به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# فقط ستون‌های موردنیاز رو برمی‌داریم
train = train[useful_features + categorical_cols + ['satisfaction']]
test = test[useful_features + categorical_cols]

In [49]:
train['satisfaction'].head()

In [50]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [51]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# # 1- Service Quality Score
# train['service_score'] = (
#     train['Inflight entertainment'] +
#     train['Seat comfort'] +
#     train['On-board service'] +
#     train['Leg room service'] +
#     train['Cleanliness'] +
#     train['Inflight wifi service'] +
#     train['Inflight service'] +
#     train['Checkin service'] +
#     train['Food and drink']
# )
# test['service_score'] = (
#     test['Inflight entertainment'] +
#     test['Seat comfort'] +
#     test['On-board service'] +
#     test['Leg room service'] +
#     test['Cleanliness'] +
#     test['Inflight wifi service'] +
#     test['Inflight service'] +
#     test['Checkin service'] +
#     test['Food and drink']
# )

In [52]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [53]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [54]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [56]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [57]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [58]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [59]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [60]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# # 1- Service Quality Score
# train['service_score'] = (
#     train['Inflight entertainment'] +
#     train['Seat comfort'] +
#     train['On-board service'] +
#     train['Leg room service'] +
#     train['Cleanliness'] +
#     train['Inflight wifi service'] +
#     train['Inflight service'] +
#     train['Checkin service'] +
#     train['Food and drink']
# )
# test['service_score'] = (
#     test['Inflight entertainment'] +
#     test['Seat comfort'] +
#     test['On-board service'] +
#     test['Leg room service'] +
#     test['Cleanliness'] +
#     test['Inflight wifi service'] +
#     test['Inflight service'] +
#     test['Checkin service'] +
#     test['Food and drink']
# )

In [61]:
train['satisfaction'].head()

In [62]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [63]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [64]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [65]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [66]:
test.head()

In [67]:
train.isna().sum()

In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [71]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# فقط ستون‌های مؤثر رو نگه می‌داریم:
useful_features = [
    'Online boarding',
    'Inflight entertainment',
    'Seat comfort',
    'On-board service',
    'Leg room service',
    'Cleanliness',
    'Flight Distance',
    'Inflight wifi service',
    'Baggage handling',
    'Inflight service',
    'Checkin service',
    'Food and drink',
    'Ease of Online booking',
    'Age',
    'service_score',
    'Travel_Class',
    'Is_New_Customer',
    'Flight Category'
]

# به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# فقط ستون‌های موردنیاز رو برمی‌داریم
train = train[useful_features + categorical_cols + ['satisfaction']]
test = test[useful_features + categorical_cols]

In [74]:
# !pip install keras --upgrade
# uncomment the above line if you don't have keras 3 installed
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [75]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [76]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [77]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [78]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# # 1- Service Quality Score
# train['service_score'] = (
#     train['Inflight entertainment'] +
#     train['Seat comfort'] +
#     train['On-board service'] +
#     train['Leg room service'] +
#     train['Cleanliness'] +
#     train['Inflight wifi service'] +
#     train['Inflight service'] +
#     train['Checkin service'] +
#     train['Food and drink']
# )
# test['service_score'] = (
#     test['Inflight entertainment'] +
#     test['Seat comfort'] +
#     test['On-board service'] +
#     test['Leg room service'] +
#     test['Cleanliness'] +
#     test['Inflight wifi service'] +
#     test['Inflight service'] +
#     test['Checkin service'] +
#     test['Food and drink']
# )

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# فقط ستون‌های مؤثر رو نگه می‌داریم:
useful_features = [
    'Online boarding',
    'Inflight entertainment',
    'Seat comfort',
    'On-board service',
    'Leg room service',
    'Cleanliness',
    'Flight Distance',
    'Inflight wifi service',
    'Baggage handling',
    'Inflight service',
    'Checkin service',
    'Food and drink',
    'Ease of Online booking',
    'Age',
    'service_score',
    'Travel_Class',
    'Is_New_Customer',
    'Flight Category'
]

# به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# فقط ستون‌های موردنیاز رو برمی‌داریم
train = train[useful_features + categorical_cols + ['satisfaction']]
test = test[useful_features + categorical_cols]

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# # فقط ستون‌های مؤثر رو نگه می‌داریم:
# useful_features = [
#     'Online boarding',
#     'Inflight entertainment',
#     'Seat comfort',
#     'On-board service',
#     'Leg room service',
#     'Cleanliness',
#     'Flight Distance',
#     'Inflight wifi service',
#     'Baggage handling',
#     'Inflight service',
#     'Checkin service',
#     'Food and drink',
#     'Ease of Online booking',
#     'Age',
#     'service_score',
#     'Travel_Class',
#     'Is_New_Customer',
#     'Flight Category'
# ]

# # به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
# categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# # فقط ستون‌های موردنیاز رو برمی‌داریم
# train = train[useful_features + categorical_cols + ['satisfaction']]
# test = test[useful_features + categorical_cols]

In [81]:
train['satisfaction'].head()

In [82]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [83]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [84]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [85]:
test.head()

In [86]:
train.isna().sum()

In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [88]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# 1- Service Quality Score
train['service_score'] = (
    train['Inflight entertainment'] +
    train['Seat comfort'] +
    train['On-board service'] +
    train['Leg room service'] +
    train['Cleanliness'] +
    train['Inflight wifi service'] +
    train['Inflight service'] +
    train['Checkin service'] +
    train['Food and drink']
)
test['service_score'] = (
    test['Inflight entertainment'] +
    test['Seat comfort'] +
    test['On-board service'] +
    test['Leg room service'] +
    test['Cleanliness'] +
    test['Inflight wifi service'] +
    test['Inflight service'] +
    test['Checkin service'] +
    test['Food and drink']
)

In [89]:
#2-Total Delay
train['Total Delay'] = train['Departure Delay in Minutes'] + train['Arrival Delay in Minutes']
train['Delay Category'] = pd.cut(train['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

#2-Total Delay
test['Total Delay'] = test['Departure Delay in Minutes'] + test['Arrival Delay in Minutes']
test['Delay Category'] = pd.cut(test['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

In [90]:
# 3- Travel Type
train['Travel_Class'] = train['Type of Travel']  + train['Class']
test['Travel_Class'] = test['Type of Travel']  + test['Class']

In [91]:
#2-Total Delay
train['Total Delay'] = train['Departure Delay in Minutes'] + train['Arrival Delay in Minutes']
train['Delay Category'] = pd.cut(train['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

#2-Total Delay
test['Total Delay'] = test['Departure Delay in Minutes'] + test['Arrival Delay in Minutes']
test['Delay Category'] = pd.cut(test['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

In [92]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [93]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [94]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [95]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# 1- Service Quality Score
train['service_score'] = (
    train['Inflight entertainment'] +
    train['Seat comfort'] +
    train['On-board service'] +
    train['Leg room service'] +
    train['Cleanliness'] +
    train['Inflight wifi service'] +
    train['Inflight service'] +
    train['Checkin service'] +
    train['Food and drink']
)
test['service_score'] = (
    test['Inflight entertainment'] +
    test['Seat comfort'] +
    test['On-board service'] +
    test['Leg room service'] +
    test['Cleanliness'] +
    test['Inflight wifi service'] +
    test['Inflight service'] +
    test['Checkin service'] +
    test['Food and drink']
)

In [96]:
#2-Total Delay
train['Total Delay'] = train['Departure Delay in Minutes'] + train['Arrival Delay in Minutes']
train['Delay Category'] = pd.cut(train['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

#2-Total Delay
test['Total Delay'] = test['Departure Delay in Minutes'] + test['Arrival Delay in Minutes']
test['Delay Category'] = pd.cut(test['Total Delay'], bins=[-1, 0, 15, 60, 180, float('inf')],
                               labels=['No delay', 'Short', 'Moderate', 'Long', 'Extreme'])

In [97]:
# 3- Travel Type
train['Travel_Class'] = train['Type of Travel']  + train['Class']
test['Travel_Class'] = test['Type of Travel']  + test['Class']

In [98]:
# 4- Loyalty or New Customer
train['Is_New_Customer'] = (train['Customer Type'] == 'disloyal Customer').astype(int)
test['Is_New_Customer'] = (test['Customer Type'] == 'disloyal Customer').astype(int)

In [99]:
# 5- Flight Duration
train['Flight Category'] = pd.cut(train['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])
test['Flight Category'] = pd.cut(test['Flight Distance'], bins=[0, 500, 1500, 3000, float('inf')],
                               labels=['Short', 'Medium', 'Long', 'Very Long'])

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# # فقط ستون‌های مؤثر رو نگه می‌داریم:
# useful_features = [
#     'Online boarding',
#     'Inflight entertainment',
#     'Seat comfort',
#     'On-board service',
#     'Leg room service',
#     'Cleanliness',
#     'Flight Distance',
#     'Inflight wifi service',
#     'Baggage handling',
#     'Inflight service',
#     'Checkin service',
#     'Food and drink',
#     'Ease of Online booking',
#     'Age',
#     'service_score',
#     'Travel_Class',
#     'Is_New_Customer',
#     'Flight Category'
# ]

# # به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
# categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# # فقط ستون‌های موردنیاز رو برمی‌داریم
# train = train[useful_features + categorical_cols + ['satisfaction']]
# test = test[useful_features + categorical_cols]

In [101]:
train['satisfaction'].head()

In [102]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [103]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [104]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [105]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Ensure all columns in X_train and X_valid are numeric
X_train = X_train.select_dtypes(include=np.number)
X_valid = X_valid.select_dtypes(include=np.number)

# If y_train is not numeric (e.g., 'satisfied'/'dissatisfied'), encode it:
if y_train.dtype == 'object':
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_valid = label_encoder.transform(y_valid)  # Use the same encoder for validation


# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [107]:
train = pd.read_csv('train.csv',engine='pyarrow')
test = pd.read_csv('test.csv',engine ='pyarrow')

# TODO

In [108]:
train= train.drop(['Unnamed: 0','id'],axis=1)
test= test.drop(['Unnamed: 0','id'],axis=1)

In [109]:
#train.isna().any()
train['Arrival Delay in Minutes'].fillna(train['Arrival Delay in Minutes'].mean(), inplace=True)

test['Arrival Delay in Minutes'].fillna(test['Arrival Delay in Minutes'].mean(), inplace=True)
train.isna().sum()

In [110]:
# #Feature Engineering

# normilzation
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Age', 'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes'
]

# مقیاس‌دهی 0 تا 1
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
# # 1- Service Quality Score
# train['service_score'] = (
#     train['Inflight entertainment'] +
#     train['Seat comfort'] +
#     train['On-board service'] +
#     train['Leg room service'] +
#     train['Cleanliness'] +
#     train['Inflight wifi service'] +
#     train['Inflight service'] +
#     train['Checkin service'] +
#     train['Food and drink']
# )
# test['service_score'] = (
#     test['Inflight entertainment'] +
#     test['Seat comfort'] +
#     test['On-board service'] +
#     test['Leg room service'] +
#     test['Cleanliness'] +
#     test['Inflight wifi service'] +
#     test['Inflight service'] +
#     test['Checkin service'] +
#     test['Food and drink']
# )

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import pandas as pd
from keras import regularizers

# # فقط ستون‌های مؤثر رو نگه می‌داریم:
# useful_features = [
#     'Online boarding',
#     'Inflight entertainment',
#     'Seat comfort',
#     'On-board service',
#     'Leg room service',
#     'Cleanliness',
#     'Flight Distance',
#     'Inflight wifi service',
#     'Baggage handling',
#     'Inflight service',
#     'Checkin service',
#     'Food and drink',
#     'Ease of Online booking',
#     'Age',
#     'service_score',
#     'Travel_Class',
#     'Is_New_Customer',
#     'Flight Category'
# ]

# # به‌علاوه ستون‌های دسته‌ای که باید تبدیل بشن:
# categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# # فقط ستون‌های موردنیاز رو برمی‌داریم
# train = train[useful_features + categorical_cols + ['satisfaction']]
# test = test[useful_features + categorical_cols]

In [112]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add target back to the feature DataFrame temporarily
df_corr = train.copy()
df_corr['satisfaction_encoded'] = LabelEncoder().fit_transform(df_corr['satisfaction'])

# Get numeric correlation
corr = df_corr.corr(numeric_only=True)

# Correlation with the target
target_corr = corr['satisfaction_encoded'].sort_values(ascending=False)

# Display the most and least related features
print(target_corr)

# Plot
plt.figure(figsize=(10,5))
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title("Feature Correlation with Satisfaction")
plt.show()

In [113]:
dummy_columns = train.drop('satisfaction', axis=1).select_dtypes(include=['object', 'category']).columns
print(dummy_columns)


train = pd.get_dummies(train, columns=dummy_columns,dtype=np.uint8)
test = pd.get_dummies(test, columns=dummy_columns,dtype=np.uint8)

In [114]:
# TODO: Make Validation Set (Optional)


from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop('satisfaction', axis=1),  # drop the column by name
    train['satisfaction'],               # target variable
    test_size=0.3,
    random_state=42                     # optional: for reproducibility
)

In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))
L2_REG = 1e-5

x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.L1L2(L2_REG))(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.95, min_lr=1e-5, verbose=1)

# Ensure all columns in X_train and X_valid are numeric
X_train = X_train.select_dtypes(include=np.number)
X_valid = X_valid.select_dtypes(include=np.number)

# If y_train is not numeric (e.g., 'satisfied'/'dissatisfied'), encode it:
if y_train.dtype == 'object':
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_valid = label_encoder.transform(y_valid)  # Use the same encoder for validation


# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,  # Increased epochs
    batch_size=32,  # Reduced batch size
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [116]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(256, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(64, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Step 1: Encode categorical features using one-hot encoding (drop_first to avoid dummy trap)
X = pd.get_dummies(train.drop(columns=['satisfaction']), drop_first=True)

# Step 2: Encode target variable
y = LabelEncoder().fit_transform(train['satisfaction'])  # 0: neutral/dissatisfied, 1: satisfied

# Step 3: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Define the model with best hyperparameters
inputs = Input(shape=(X_train.shape[1],))

# First Dense block: 320 neurons with dropout 0.3
x = Dense(320, kernel_initializer='he_normal')(inputs)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Second Dense block: 160 neurons with dropout 0.3
x = Dense(160, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Third Dense block: 96 neurons with dropout 0.1
x = Dense(96, kernel_initializer='he_normal')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

# Output layer for binary classification
outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Step 6: Compile model using the specified learning rate (0.001)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 7: Define callbacks
checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True)
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5, verbose=1)

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [118]:
from sklearn.metrics import accuracy_score

# Evaluate on the validation set
y_val_pred_probs = model.predict(X_valid)
y_val_pred = (y_val_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_valid, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Predict on the test set (assumes 'test' is preprocessed identically to the train set)
y_test_pred_probs = model.predict(test)
y_test_pred = (y_test_pred_probs > 0.5).astype(int).flatten()

# Map predicted classes (0/1) to labels
label_map = {0: 'neutral or dissatisfied', 1: 'satisfied'}
y_test_labels = [label_map[val] for val in y_test_pred]
print("Test set predictions:")
print(y_test_labels)

In [119]:
submission = pd.DataFrame()
submission['satisfaction'] = y_test_labels
print(submission)